In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import date
import requests
from minio import Minio
from io import BytesIO
from OCR_app import Model


load_dotenv()

# Define constant
URL = os.environ.get("URL")
USER_NAME = os.environ.get("USER_NAME")
PASSWORD = os.environ.get("PASSWORD")
HOST = os.environ.get("HOST")
PORT = 5432
DATABASE = os.environ.get("DATABASE") 

MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY")

BUCKET_NAME = "datalake"
ENDPOINT = "localhost:9000"

minio_bucket = "datalake"

minio_client = Minio(ENDPOINT,
                      access_key=MINIO_ACCESS_KEY,
                      secret_key=MINIO_SECRET_KEY,
                      secure=False)


### Read image using OCR and parse it to pandas DataFrame

In [5]:
list_obj = minio_client.list_objects("datalake", prefix="PaperNote", recursive=True)

dateTime_templates = ['Accepted at', 'Completed at', 'Boarded at', 'Picked up at']
customer_templates = ['cusName', 'Address', 'Phone', 'Email']
order_templates = ['ordName', 'Price', 'Trip type']
staff_templates = ['staffName', 'Phone', 'Email', "StaffID", "Store"]
model = Model()
columns = dateTime_templates + customer_templates + order_templates + staff_templates
df_papernote = pd.DataFrame(columns=columns)
count = 0
for obj in list_obj:
    img = obj.object_name
    response = minio_client.get_object(BUCKET_NAME, img)
    model.load_image(response=response)
    json = model.detect_ocr()
    json_flat = {}
    for i in json:
        json_flat.update(i)
    ls = [json_flat.get(i+':') for i in columns]
    df_papernote.loc[len(df_papernote.index)] = ls

    print("Done",obj.object_name)
df_papernote.head()

[2024/04/17 08:12:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\admin/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\admin/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_batch_n

KeyboardInterrupt: 

In [9]:
df_papernote.drop(['Address', 'Phone', 'Email', 'ordName', 'staffName', 'Phone', 'Email', 'cusName'], axis=1).head()

,Accepted at,Completed at,Boarded at,Picked up at,Price,Trip type,StaffID,Store
0,12:15PM 2024-02-19,03:15PM 2024-02-19,04:59PM 2024-02-19,06:27PM 2024-02-19,$82.95,Round,SD26,Fresh Fare
1,05:40PM 2024-04-26,09:40PM 2024-04-26,11:06PM 2024-04-26,12:14AM 2024-04-27,$45.32,Single,SD01,Corner Market
2,10:56AM 2024-02-07,02:56PM 2024-02-07,04:49PM 2024-02-07,05:55PM 2024-02-07,$81.01,Round,SD30,Fresh Fare
3,02:53PM 2024-02-03,05:53PM 2024-02-03,07:19PM 2024-02-03,09:07PM 2024-02-03,$29.82,Round,SD07,Budget Mart


### Preprocessing DataFrame

In [105]:
for i in dateTime_templates:
    df_papernote[i] = pd.to_datetime(df_papernote[i])
df_papernote['Price'] = pd.to_numeric(df_papernote['Price'].str[1:])
df_papernote["ShipType"] = df_papernote['Trip type'].apply(lambda x: 0 if x == "Round" else 1)
df_papernote = df_papernote.drop("Trip type", axis=1)

In [10]:
def read_data(table):
    print("Read data from {table}".format(table=table))
    df = pd.read_parquet(f"s3://datalake/{table}".format(table=table),
                    storage_options={
                        "key": MINIO_ACCESS_KEY,
                        "secret": MINIO_SECRET_KEY,
                        "client_kwargs": {"endpoint_url": "http://localhost:9000/"}
                    }).drop(['year', 'month', 'day'], axis=1)
    return df

In [11]:
df_product = read_data(table="Product")
df_store = read_data(table="Store")
df_account = read_data(table="Account")
df_order = read_data(table="Order")
df_shipping = read_data(table="Shipping")
df_shippingservice = read_data(table="ShippingService")

Read data from Product
Read data from Store
Read data from Account
Read data from Order
Read data from Shipping
Read data from ShippingService


### Create connection to Trino and schema

In [ ]:
from trino.dbapi import connect
conn = connect(
    host="localhost",
    port=8080,
    user="admin",
    catalog="iceberg",
)
cur = conn.cursor()
cur.execute("show schemas")
cur.fetchall()
cur.execute(f"CREATE SCHEMA IF NOT EXISTS iceberg.shipping_report with (LOCATION = 's3a://lakehouse/shipping_report/')")

### CREATE TABLE CANCEL TABLE

In [20]:
df = df_shipping.merge(df_shippingservice, how="inner", left_on="serviceid", right_on="id")
df = df[["order_id", "cancelled_at", "servicename"]]
df = df.merge(df_order[["id","storeid"]], left_on="order_id", right_on="id")
df = df.merge(df_store, left_on="storeid", right_on="id")


In [22]:
df = df.drop(["id_x", "id_y", "storeid"], axis=1)
df = df.dropna(axis=0)


In [26]:
df.cancelled_at = df['cancelled_at'].dt.time

In [27]:
df.head()

,order_id,cancelled_at,servicename,street,storename,city,stt
1,23a4b5c6d-01f2-3456-789a-ef0123456789,13:00:00,UPS Next Day Air,666 Cedar St,Discount Depot,Fort Worth,13
9,26d7e8f9a-3456-789a-bcde-123456789abc,13:00:00,FedEx Express Saver,890 Birch St,Fresh Fare,San Antonio,7
27,17a8b9c0d-789a-bcde-0123-789abcdef012,13:00:00,FedEx Ground,567 Maple St,Neighborhood Market,Philadelphia,6
28,25c6d7e8f-2345-6789-abcd-0123456789ab,13:00:00,FedEx Ground,444 Elm St,Value Foods,Austin,11
44,d76f83d8-e4a2-4f77-bcde-2480a04246f1,15:00:00,USPS Priority Mail,890 Birch St,Fresh Fare,San Antonio,7


In [30]:
df = df.drop("stt", axis=1)

In [32]:
query = """CREATE TABLE IF NOT EXISTS iceberg.shipping_report.order_by_service(
    order_id varchar,
    cancelled_at time(6),
    servicename varchar,
    street varchar,
    storename varchar,
    city varchar
)"""
cur.execute(query)

In [34]:
for i in range(len(df)):
    order_id = df.iloc[i]["order_id"]
    cancelled_at = df.iloc[i]["cancelled_at"]
    servicename = df.iloc[i]["servicename"]
    street = df.iloc[i]["street"]
    storename = df.iloc[i]["storename"]
    city = df.iloc[i]["city"]
    
    
    query = f"INSERT INTO iceberg.shipping_report.order_by_service VALUES ('{order_id}',TIME '{cancelled_at}', '{servicename}', '{street}', '{storename}', '{city}')"
    cur.execute(query)
        

cur.close()

In [112]:
df_papernote = pd.read_csv("data.csv")

In [113]:
df_papernote.head()

,Unnamed: 0,Accepted at,Completed at,Boarded at,Picked up at,cusName,Address,Phone,Email,ordName,Price,staffName,Phone.1,Email.1,StaffID,Store,ShipType
0,0,2024-02-19 12:15:00,2024-02-19 15:15:00,2024-02-19 16:59:00,2024-02-19 18:27:00,Michael Perez,2166 Christine Keys,777-888-9999,emily.smith@example.com,eat hope,82.95,Emily Smith,777-888-9999,emily.smith@example.com,SD26,Fresh Fare,0
1,1,2024-04-26 17:40:00,2024-04-26 21:40:00,2024-04-26 23:06:00,2024-04-27 00:14:00,Shawn Walker,368 Jones Junctions Suite 338,555-444-3333,emily.cobb@example.com,artist size,45.32,Emily Cobb,555-444-3333,emily.cobb@example.com,SD01,Corner Market,1
2,2,2024-02-07 10:56:00,2024-02-07 14:56:00,2024-02-07 16:49:00,2024-02-07 17:55:00,Catherine Taylor,32683 Rodriguez Squares,999-888-7777,emma.jones@example.com,attention green,81.01,Emma Jones,999-888-7777,emma.jones@example.com,SD30,Fresh Fare,0
3,3,2024-02-03 14:53:00,2024-02-03 17:53:00,2024-02-03 19:19:00,2024-02-03 21:07:00,Lori Hall,80219 Sandra Neck Suite 832,444-444-4444,jessica.bass@example.com,rock information,29.82,Jessica Bass,444-444-4444,jessica.bass@example.com,SD07,Budget Mart,0
4,4,2024-04-25 21:26:00,2024-04-26 01:26:00,2024-04-26 03:22:00,2024-04-26 04:05:00,David Mathews,948 Parker Bridge Apt. 823,555-555-5555,darren.jackson@example.com,hot down,35.89,Darren Jackson,555-555-5555,darren.jackson@example.com,SD10,Fresh Fare,1


In [80]:
df = df_shipping[df_shipping['completed_at'].notnull()]
df = df.drop("cancelled_at", axis=1)


In [114]:
df_papernote.rename(columns={'Accepted at': 'accepted_at', 
                             'Completed at': 'boarded_at',
                             'Boarded at': 'picked_up_at',
                             'Picked up at': 'completed_at',
                             'ShipType': 'shiptype'}, inplace=True)

In [81]:
df = df[['accepted_at', 'boarded_at', 'picked_up_at', 
         'completed_at', 'shiptype', 'order_id', 'serviceid']]
df = df.merge(df_shippingservice, how="inner", left_on="serviceid", right_on="id")
df = df.drop(["serviceid", "serviceprovider", "stt"], axis=1)


In [82]:
df = df.merge(df_order, how="inner", left_on="order_id", right_on='id')
df.head()


,accepted_at,boarded_at,picked_up_at,completed_at,shiptype,order_id,id_x,servicename,id_y,customername,storeid,created,stt,orderdetail,price
0,2024-03-01 10:00:00,2024-03-01 11:00:00,2024-03-01 12:00:00,2024-03-01 13:00:00,0,22f3a4b5c-c01f-2345-6789-def012345678,4b3e1f73-40b2-4819-a03c-ce00cb8a5f2f,UPS Ground,22f3a4b5c-c01f-2345-6789-def012345678,Matthew Moore,ea2871e3-fcd7-4f3c-8ef7-2b4aaf1cfd56,2024-04-06 13:45:00,33,detail 33,78
1,2024-03-03 10:00:00,2024-03-03 11:00:00,2024-03-03 12:00:00,2024-03-03 13:00:00,1,24b5c6d7e-1f23-4567-89ab-f0123456789a,a25a0eea-0e38-468b-9d9a-36db621b482a,UPS 2nd Day Air,24b5c6d7e-1f23-4567-89ab-f0123456789a,John Davis,f35a7e6c-cb76-4a13-92cf-d3c59a1e01c3,2024-04-06 15:15:00,35,detail 35,89
2,2024-03-06 10:00:00,2024-03-06 11:00:00,2024-03-06 12:00:00,2024-03-06 13:00:00,1,27e8f9a0b-4567-89ab-cdef-23456789abcd,6248a0e5-cc9a-40be-8695-045da8c5425d,FedEx 2Day,27e8f9a0b-4567-89ab-cdef-23456789abcd,Mia Moore,20d23aa0-304d-4b4a-a08e-5e2c98c69b57,2024-04-06 18:00:00,38,detail 38,3
3,2024-03-07 10:00:00,2024-03-07 11:00:00,2024-03-07 12:00:00,2024-03-07 13:00:00,1,28f9a0b1c-5678-9abc-def0-3456789abcde,79aa7af9-40e7-43d8-9f1c-83e09fc9f004,DHL Express Worldwide,28f9a0b1c-5678-9abc-def0-3456789abcde,Noah Garcia,2792a53e-fad5-4a9d-b2b0-39ac0bb1c77a,2024-04-06 19:15:00,39,detail 39,91
4,2024-03-08 10:00:00,2024-03-08 11:00:00,2024-03-08 12:00:00,2024-03-08 13:00:00,0,29a0b1c2d-6789-abcd-ef01-456789abcdef,eb7983a3-2f65-4113-a391-d9114143e402,DHL Express Domestic,29a0b1c2d-6789-abcd-ef01-456789abcdef,Grace Wilson,2fd747fd-8be8-4b0c-b2ad-62d4765de4a2,2024-04-06 20:30:00,40,detail 40,44


In [83]:
df = df.drop(['order_id', 'id_x', 'id_y', 'customername', 'created', 'stt', 'orderdetail'], axis=1)

In [90]:
df = df.merge(df_store, left_on="storeid", right_on="id")
df.head()

,accepted_at,boarded_at,picked_up_at,completed_at,shiptype,servicename,storeid,price,id,street,storename,city,stt
0,2024-03-01 10:00:00,2024-03-01 11:00:00,2024-03-01 12:00:00,2024-03-01 13:00:00,0,UPS Ground,ea2871e3-fcd7-4f3c-8ef7-2b4aaf1cfd56,78,ea2871e3-fcd7-4f3c-8ef7-2b4aaf1cfd56,555 Pine St,Bulk Bargains,Jacksonville,12
1,2024-03-03 10:00:00,2024-03-03 11:00:00,2024-03-03 12:00:00,2024-03-03 13:00:00,1,UPS 2nd Day Air,f35a7e6c-cb76-4a13-92cf-d3c59a1e01c3,89,f35a7e6c-cb76-4a13-92cf-d3c59a1e01c3,777 Maple St,Super Savings,Columbus,14
2,2024-03-06 10:00:00,2024-03-06 11:00:00,2024-03-06 12:00:00,2024-03-06 13:00:00,1,FedEx 2Day,20d23aa0-304d-4b4a-a08e-5e2c98c69b57,3,20d23aa0-304d-4b4a-a08e-5e2c98c69b57,789 Elm St,BigBox Store,Chicago,3
3,2024-03-07 10:00:00,2024-03-07 11:00:00,2024-03-07 12:00:00,2024-03-07 13:00:00,1,DHL Express Worldwide,2792a53e-fad5-4a9d-b2b0-39ac0bb1c77a,91,2792a53e-fad5-4a9d-b2b0-39ac0bb1c77a,123 Main St,SuperMart,New York,1
4,2024-03-08 10:00:00,2024-03-08 11:00:00,2024-03-08 12:00:00,2024-03-08 13:00:00,0,DHL Express Domestic,2fd747fd-8be8-4b0c-b2ad-62d4765de4a2,44,2fd747fd-8be8-4b0c-b2ad-62d4765de4a2,456 Oak St,MegaMart,Los Angeles,2


In [91]:
df = df.drop(['storeid', 'id', 'street', 'city', 'stt'], axis=1)
df.head()

,accepted_at,boarded_at,picked_up_at,completed_at,shiptype,servicename,price,storename
0,2024-03-01 10:00:00,2024-03-01 11:00:00,2024-03-01 12:00:00,2024-03-01 13:00:00,0,UPS Ground,78,Bulk Bargains
1,2024-03-03 10:00:00,2024-03-03 11:00:00,2024-03-03 12:00:00,2024-03-03 13:00:00,1,UPS 2nd Day Air,89,Super Savings
2,2024-03-06 10:00:00,2024-03-06 11:00:00,2024-03-06 12:00:00,2024-03-06 13:00:00,1,FedEx 2Day,3,BigBox Store
3,2024-03-07 10:00:00,2024-03-07 11:00:00,2024-03-07 12:00:00,2024-03-07 13:00:00,1,DHL Express Worldwide,91,SuperMart
4,2024-03-08 10:00:00,2024-03-08 11:00:00,2024-03-08 12:00:00,2024-03-08 13:00:00,0,DHL Express Domestic,44,MegaMart


In [115]:
df_papernote.head()

,Unnamed: 0,accepted_at,boarded_at,picked_up_at,completed_at,cusName,Address,Phone,Email,ordName,Price,staffName,Phone.1,Email.1,StaffID,Store,shiptype
0,0,2024-02-19 12:15:00,2024-02-19 15:15:00,2024-02-19 16:59:00,2024-02-19 18:27:00,Michael Perez,2166 Christine Keys,777-888-9999,emily.smith@example.com,eat hope,82.95,Emily Smith,777-888-9999,emily.smith@example.com,SD26,Fresh Fare,0
1,1,2024-04-26 17:40:00,2024-04-26 21:40:00,2024-04-26 23:06:00,2024-04-27 00:14:00,Shawn Walker,368 Jones Junctions Suite 338,555-444-3333,emily.cobb@example.com,artist size,45.32,Emily Cobb,555-444-3333,emily.cobb@example.com,SD01,Corner Market,1
2,2,2024-02-07 10:56:00,2024-02-07 14:56:00,2024-02-07 16:49:00,2024-02-07 17:55:00,Catherine Taylor,32683 Rodriguez Squares,999-888-7777,emma.jones@example.com,attention green,81.01,Emma Jones,999-888-7777,emma.jones@example.com,SD30,Fresh Fare,0
3,3,2024-02-03 14:53:00,2024-02-03 17:53:00,2024-02-03 19:19:00,2024-02-03 21:07:00,Lori Hall,80219 Sandra Neck Suite 832,444-444-4444,jessica.bass@example.com,rock information,29.82,Jessica Bass,444-444-4444,jessica.bass@example.com,SD07,Budget Mart,0
4,4,2024-04-25 21:26:00,2024-04-26 01:26:00,2024-04-26 03:22:00,2024-04-26 04:05:00,David Mathews,948 Parker Bridge Apt. 823,555-555-5555,darren.jackson@example.com,hot down,35.89,Darren Jackson,555-555-5555,darren.jackson@example.com,SD10,Fresh Fare,1


In [116]:
df_papernote = df_papernote.drop(['Unnamed: 0', 'cusName', 'Address', 'Phone', 'Email', 'ordName',
                                   'staffName', 'Phone.1', 'Email.1'], axis=1)

In [120]:
df_papernote = df_papernote.drop('Unnamed: 0', axis=1)

In [121]:
df_papernote.head()

,accepted_at,boarded_at,picked_up_at,completed_at,price,StaffID,storename,shiptype
0,2024-02-19 12:15:00,2024-02-19 15:15:00,2024-02-19 16:59:00,2024-02-19 18:27:00,82.95,SD26,Fresh Fare,0
1,2024-04-26 17:40:00,2024-04-26 21:40:00,2024-04-26 23:06:00,2024-04-27 00:14:00,45.32,SD01,Corner Market,1
2,2024-02-07 10:56:00,2024-02-07 14:56:00,2024-02-07 16:49:00,2024-02-07 17:55:00,81.01,SD30,Fresh Fare,0
3,2024-02-03 14:53:00,2024-02-03 17:53:00,2024-02-03 19:19:00,2024-02-03 21:07:00,29.82,SD07,Budget Mart,0
4,2024-04-25 21:26:00,2024-04-26 01:26:00,2024-04-26 03:22:00,2024-04-26 04:05:00,35.89,SD10,Fresh Fare,1


In [118]:
df_papernote.rename(columns={"Price": "price", 'Store': "storename"}, inplace=True)

In [122]:
shipping_report = pd.concat([df, df_papernote], join="outer", axis=0)

In [123]:
shipping_report.head()

,accepted_at,boarded_at,picked_up_at,completed_at,shiptype,servicename,price,storename,StaffID
0,2024-03-01 10:00:00,2024-03-01 11:00:00,2024-03-01 12:00:00,2024-03-01 13:00:00,0,UPS Ground,78,Bulk Bargains,NaN
1,2024-03-03 10:00:00,2024-03-03 11:00:00,2024-03-03 12:00:00,2024-03-03 13:00:00,1,UPS 2nd Day Air,89,Super Savings,NaN
2,2024-03-06 10:00:00,2024-03-06 11:00:00,2024-03-06 12:00:00,2024-03-06 13:00:00,1,FedEx 2Day,3,BigBox Store,NaN
3,2024-03-07 10:00:00,2024-03-07 11:00:00,2024-03-07 12:00:00,2024-03-07 13:00:00,1,DHL Express Worldwide,91,SuperMart,NaN
4,2024-03-08 10:00:00,2024-03-08 11:00:00,2024-03-08 12:00:00,2024-03-08 13:00:00,0,DHL Express Domestic,44,MegaMart,NaN


In [129]:
for i in shipping_report.columns[:4]:
    shipping_report[i] = pd.to_datetime(shipping_report[i])

In [152]:
shipping_report['duration'] = abs((shipping_report['completed_at'] - shipping_report['boarded_at']).dt.total_seconds() / 3600)


In [135]:
shipping_report['servicename'] = shipping_report['servicename'].fillna("Staff")
shipping_report['StaffID'] = shipping_report['StaffID'].fillna("Service")

In [147]:
shipping_report.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309 entries, 0 to 231
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   accepted_at   309 non-null    datetime64[ns]
 1   boarded_at    309 non-null    datetime64[ns]
 2   picked_up_at  309 non-null    datetime64[ns]
 3   completed_at  309 non-null    datetime64[ns]
 4   shiptype      309 non-null    int64         
 5   servicename   309 non-null    object        
 6   price         309 non-null    object        
 7   storename     309 non-null    object        
 8   StaffID       309 non-null    object        
 9   duration      309 non-null    float64       
dtypes: datetime64[ns](4), float64(1), int64(1), object(4)
memory usage: 26.6+ KB


In [153]:
cur.execute("drop table iceberg.shipping_report.shipping")

In [154]:
query = """CREATE TABLE IF NOT EXISTS iceberg.shipping_report.shipping(
    accepted_at timestamp(6),
    boarded_at timestamp(6),
    picked_up_at timestamp(6),
    completed_at timestamp(6),
    shiptype int,
    servicename varchar,
    price real,
    storename varchar,
    StaffID varchar,
    duration real
)"""
cur.execute(query)

In [155]:
for i in range(len(shipping_report)):
    accepted_at = shipping_report.iloc[i]["accepted_at"]
    boarded_at = shipping_report.iloc[i]["boarded_at"]
    picked_up_at = shipping_report.iloc[i]["picked_up_at"]
    completed_at = shipping_report.iloc[i]["completed_at"]

    shiptype = shipping_report.iloc[i]["shiptype"]
    servicename = shipping_report.iloc[i]["servicename"]
    price = shipping_report.iloc[i]["price"]
    storename = shipping_report.iloc[i]["storename"]

    StaffID = shipping_report.iloc[i]["StaffID"]
    duration = shipping_report.iloc[i]["duration"]
    

    
    
    query = f"INSERT INTO iceberg.shipping_report.shipping VALUES (TIMESTAMP '{accepted_at}',TIMESTAMP '{boarded_at}', TIMESTAMP '{picked_up_at}',TIMESTAMP '{completed_at}',{shiptype}, '{servicename}', {price}, '{storename}', '{StaffID}',{duration})"
    cur.execute(query)
        

cur.close()

In [150]:
temp = (df_shipping['completed_at'] - df_shipping['boarded_at']).dt.total_seconds() / 3600

In [156]:
df_papernote.head()

,accepted_at,boarded_at,picked_up_at,completed_at,price,StaffID,storename,shiptype
0,2024-02-19 12:15:00,2024-02-19 15:15:00,2024-02-19 16:59:00,2024-02-19 18:27:00,82.95,SD26,Fresh Fare,0
1,2024-04-26 17:40:00,2024-04-26 21:40:00,2024-04-26 23:06:00,2024-04-27 00:14:00,45.32,SD01,Corner Market,1
2,2024-02-07 10:56:00,2024-02-07 14:56:00,2024-02-07 16:49:00,2024-02-07 17:55:00,81.01,SD30,Fresh Fare,0
3,2024-02-03 14:53:00,2024-02-03 17:53:00,2024-02-03 19:19:00,2024-02-03 21:07:00,29.82,SD07,Budget Mart,0
4,2024-04-25 21:26:00,2024-04-26 01:26:00,2024-04-26 03:22:00,2024-04-26 04:05:00,35.89,SD10,Fresh Fare,1


In [158]:
query = """CREATE TABLE IF NOT EXISTS iceberg.shipping_report.paper_note(
    accepted_at timestamp(6),
    boarded_at timestamp(6),
    picked_up_at timestamp(6),
    completed_at timestamp(6),
    price real,
    StaffID varchar,
    storename varchar,
    shiptype int)
    """
cur.execute(query)

In [160]:
for i in range(len(df_papernote)):
    accepted_at = df_papernote.iloc[i]["accepted_at"]
    boarded_at = df_papernote.iloc[i]["boarded_at"]
    picked_up_at = df_papernote.iloc[i]["picked_up_at"]
    completed_at = df_papernote.iloc[i]["completed_at"]

    shiptype = df_papernote.iloc[i]["shiptype"]
    price = df_papernote.iloc[i]["price"]
    storename = df_papernote.iloc[i]["storename"]
    StaffID = df_papernote.iloc[i]["StaffID"]

    

    
    
    query = f"INSERT INTO iceberg.shipping_report.paper_note VALUES (TIMESTAMP '{accepted_at}',TIMESTAMP '{boarded_at}', TIMESTAMP '{picked_up_at}',TIMESTAMP '{completed_at}', {price}, '{StaffID}', '{storename}',{shiptype})"
    cur.execute(query)